# <center> <u>**Scrap iNaturalist** </u></center>



# 1. Récupérer les noms latins des espèces désirées

On part d'une csv d'espèces d'abeilles extraite sur [taxref](https://inpn.mnhn.fr/accueil/index) des espèces que l'on veut scraper. 
On transforme cette csv en liste de noms latins.
Cette liste se trouve [ici](../datafiles/scrap_inat/taxref_abeilles.csv) dans notre cas. 

In [5]:
import pandas as pd
import os 

path_to_desired_taxa = '/datafiles/scrap_inat/taxref_abeilles.csv'
df_desired_taxa = pd.read_csv(path_to_desired_taxa, sep=';', encoding='latin-1')

df_desired_taxa = df_desired_taxa['LB_NOM']
df_desired_taxa = df_desired_taxa.drop_duplicates()


In [8]:
path_folder_inat_util = '../data_bees_detection/inat_utils/'

if not os.path.exists(path_folder_inat_util):
    os.makedirs(path_folder_inat_util)

# save the list of desired taxa
path_to_save = path_folder_inat_util + 'desired_taxa.csv'
# save the list of desired taxa
df_desired_taxa.to_csv(path_to_save, index=False)


On cherche donc l'ensemble des points d'observation d'abeilles dans ce périmètre.

On postule que :
rayon terre = 6371km

distance = arccos(sin(lat_cercle) * sin(lat_point) + cos(lat_cercle) * cos(lat_point) * cos(long_cercle - long_point)) * rayon_terrestre

On cherche donc les points d'observation qui sont à une distance inférieure à 500km du centre du cercle.

i.e.

arccos(sin(lat_cercle) * sin(lat_point) + cos(lat_cercle) * cos(lat_point) * cos(long_cercle - long_point)) * rayon_terrestre <= 500



* bouding box france : 'FR': ('France', (-54.5247541978, 2.05338918702, 9.56001631027, 51.1485061713))

# 2. Télécharger et filtrer les métadonnées Inaturalist

Pour ce faire, suivre le tuto décrit [ici](../datafiles/scrap_inat/download_inaturalist.md) 


# 3. Télécharger les images

## 3.1 Déterminer la taille de l'image à télécharger

In [21]:
import os
from PIL import Image
import pandas as pd



path_to_bd_71_cropped = '../data_bees_detection/Cropped_BD_71'
path_to_bd_71 = '../data_bees_detection/BD_71'

# get statistics on images size in each folder

def get_stats(folder):

    dict_sizes = {}

    for root, dirs, files in os.walk(folder):

        for file in files :

            try :
                #read the image
                img = Image.open(os.path.join(root, file))
            except :
                continue

            # get the size of the image
            width, height = img.size

            # add the size to the dict at file index
            dict_sizes[file] = [width, height]

    print('number of images: ', len(dict_sizes))

    # make a dataframe
    df = pd.DataFrame.from_dict(dict_sizes, orient='index')

    # rename columns
    df = df.rename(columns={0: "width", 1: "height"})

    return df


print('BD_71\n')
df = get_stats(path_to_bd_71)
print(df.describe())

print('Cropped_BD_71\n')
df = get_stats(path_to_bd_71_cropped)
print(df.describe())



BD_71

number of images:  103899
               width         height
count  103899.000000  103899.000000
mean      509.809074     477.016766
std       165.576518     122.910099
min       109.000000     105.000000
25%       375.000000     375.000000
50%       500.000000     500.000000
75%       500.000000     500.000000
max      3900.000000    2600.000000
Cropped_BD_71

number of images:  105603
               width         height
count  105603.000000  105603.000000
mean      216.787194     211.697054
std       125.641958     121.986794
min         1.000000       1.000000
25%       121.000000     120.000000
50%       194.000000     191.000000
75%       293.000000     280.000000
max      2245.000000    1975.000000


Conclusion : on peut télécharger les images en taille medium (500 pixels de largeur) pour les images qui ont une taille supérieure à 500 pixels de largeur.

## 3.2 Télécharger les images


In [2]:
output_folder = '../../data_bees_detection/Inaturalist_06_07'
input_file = '../../data_bees_detection/inat_utils/inat_filter.csv'
size = 'medium'

In [ ]:

%run ../src/scrap_inat/get_data_from_inat.py -i $input_file -o $output_folder -s $size